In [24]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
folder = 'sp_matrix_tag_pop'
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

In [2]:
train = np.loadtxt(os.path.join(dir_, folder, 'rating_matrix_' + file_name[:-3] + 'csv'), delimiter=',')
pd_train = pd.read_pickle(os.path.join(dir_, folder, 'train_' + file_name))
train

array([[1.5, 1.5, 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 4. , ..., 0. , 0. , 0. ]])

In [3]:
num_user = len(train)
num_track = len(train[0])
print(num_user, num_track)

218 39137


In [19]:
groups = []
group_size = 2
assigned = np.zeros(num_user, dtype=bool)
while len(groups) < int(num_user/group_size):
    group = []
    while len(group) < group_size:
        uid = int(random.random()*num_user)
        if not assigned[uid]:
            group.append(uid)
            assigned[uid] = True
    groups.append(group)

In [20]:
for group in groups:
    if len(group) != group_size:
        print('Group size error')
user_status = np.zeros(num_user)
for group in groups:
    for u in group:
        user_status[u] += 1
for i in user_status:
    if i > 1:
        print('Depulicate error')

In [21]:
groups[0]

[37, 29]

In [22]:
len(groups)

109

In [23]:
np.savetxt(os.path.join(dir_, 'groups/random/' + str(group_size)  + '.csv'), groups, delimiter=",")

In [9]:
def pearson_correlation(u1_index, u2_index):
        result = 0.0
        user1_data = train[u1_index]
        user2_data = train[u2_index]

        rx_avg = user_average_rating(user1_data)
        ry_avg = user_average_rating(user2_data)
        sxy = common_items(user1_data, user2_data)

        top_result = 0.0
        bottom_left_result = 0.0
        bottom_right_result = 0.0
        for item in sxy:
            rxs = user1_data[item]
            rys = user2_data[item]

            top_result += (rxs - rx_avg)*(rys - ry_avg)
            bottom_left_result += pow((rxs - rx_avg), 2)
            bottom_right_result += pow((rys - ry_avg), 2)
        bottom_left_result = math.sqrt(bottom_left_result)
        bottom_right_result = math.sqrt(bottom_right_result)
        
        ################################################################
        if (bottom_left_result * bottom_right_result) == 0:
            return -2, -2 # dump the data
        ################################################################
        
        result = top_result/(bottom_left_result * bottom_right_result)
        return len(sxy), result

def user_average_rating(u):
    avg_rating = 0.0
    for i in u:
        avg_rating += i
    avg_rating /= len(u) * 1.0
    return avg_rating

def common_items(u1, u2):
    result = []
    for i in range(num_track):
        if u1[i] > 0 and u2[i] > 0:
            result.append(i)
    return result

In [10]:
group_dir_ = '../data/groups/random'
train = np.loadtxt(os.path.join(dir_, 'rating_matrix.csv'), delimiter=',')

OSError: ../data/rating_matrix.csv not found.

In [11]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, 'group_' + group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

OSError: ../data/groups/random/group_2.csv not found.

In [7]:
similarities = []
for groups in groups_n:
    n = 0
    similarity = 0
    for group in groups:
        for i in range(len(groups[0])):
            for j in range(i+1, len(groups[0])):
                u1 = int(group[i])
                u2 = int(group[j])
                c, p = pearson_correlation(u1, u2)
                p = p * min(c,200) / 200
                similarity += p
                n += 1 
    similarity /= n
    similarities.append(similarity)

In [8]:
similarities

[0.17598573054968505,
 0.18137858891099298,
 0.1874218428224502,
 0.18423042032651812]

In [9]:
sum(similarities)/len(similarities)

0.1822541456524116